<a href="https://colab.research.google.com/github/rishavsen1/programming-assignment-0/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [ ]:
#Describe your answer below.
#Answer 1: 

1. The NASDAQ Traded dataset is read, where the columns are delimited by "|", and  is stored in "data" variable.<br>
2. "data_clean" stores the data where "Test Issues" are "N".<br>
3. A set of all the NASDAQ symbols is stored in "symbols"<br>
4. Similarly, The NASDAQ Other listed dataset is read, where the columns are delimited by "|", and  is stored in "data" variable.<br>
5. "data_clean" stores the data where "Test Issues" are "N".<br>
6. A set of all the NASDAQ other traded symbols is stored in "symbols"<br>


In [3]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-25 10:17:07--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-25 10:17:08--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5de92f43c92697bdae36dca2ee.dl.dropboxusercontent.com/cd/0/get/Bec8SgHKevhRyWPdBKzOVprEw7ZfMrc4d94isHt95G_r7HI1yPZSFFTR5P4vznQWjsFPjAoxRRLbroLb2hv3OK65v9Iy0j8gO6DDKqrkNxZJfYYCAXgwZ8Tx3EGcw6xuuuoqFqCpmeu7zyZ-MqE57wsO/file?dl=1# [following]
--2022-01-25 10:17:08--  https://uc5de92f43c92697bdae36dca2ee.dl.dropboxusercontent.com/cd/0/get/Bec8SgHKevhRyWPdBKzOVprEw7ZfMrc4d94isHt95G_r7HI1yPZSFFTR5P4vznQWjsFPjAoxRRLbroL

In [4]:
!ls stocks.parquet -l #downloaded file

-rw-r--r-- 1 root root 678640589 Jan 25 10:17 stocks.parquet


In [5]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 
symbols = pd.read_parquet('stocks.parquet', columns=["symbol"]) #all symbols
symbols_unique = symbols.drop_duplicates() #unique symbols

In [ ]:
data=pd.read_parquet('stocks.parquet')

In [ ]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z * changed it to (Y-X)/Z
# Sort the stocks by this return and find the top 20

In [ ]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [7]:
df = pd.read_parquet('stocks.parquet', columns=["date", "adj_close"])

In [8]:
from datetime import datetime
import numpy as np

d0 = datetime.strptime(df.iloc[0].date, '%Y-%m-%d').date()
X = df.iloc[0].adj_close
stock_start_ind = 0
ret = [np.nan for i in range(len(symbols_unique))]
count = 0
df_return = symbols_unique.copy(deep=False)
for i, rows in symbols_unique[1:].iterrows():
  if i-stock_start_ind >=1.00007:  #to remove stocks with 1 day's performance recorded only
    Y = df.iloc[i-1].adj_close
    d1 = datetime.strptime(df.iloc[i-1].date, '%Y-%m-%d').date()
    Z = d1 - d0
    Z = Z.days
    ret[count] = ((Y-X)/Z)
    d0 = datetime.strptime(df.iloc[i].date, '%Y-%m-%d').date()
    X = df.iloc[i].adj_close
  stock_start_ind = i-1
  count+=1

# to account for the last stock
if df.index[-1] - df.index[i] >= 1.00007:
  Y = df.iloc[-1].adj_close
  d1 = datetime.strptime(df.iloc[-1].date, '%Y-%m-%d').date()
  Z = d1 - d0
  Z = Z.days
  ret[len(ret)] = ((Y-X)/Z)
  
df_return['ret'] = ret

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [9]:
df_return = df_return.sort_values(by='ret', ascending=False)

print("Best returning stocks are:\nSymbol \t Return")
for i, rows in df_return[:20].iterrows():
  print(rows.symbol,"\t", rows.ret)


Best returning stocks are:
Symbol 	 Return
JE 	 2864610111595566.0
AVGOP 	 1.3786403322682796
HCP 	 0.6894442240397135
MNDY 	 0.6533999633789063
SHOP 	 0.5733899568256579
CABO 	 0.5716100522834653
DWAC 	 0.4887499917637218
DWACU 	 0.4752252080419042
GOOG 	 0.459267609201032
GOOGL 	 0.4587331857002765
NVR 	 0.43561101319905093
IOT 	 0.3891666730244954
AMZN 	 0.37716098304745144
SITM 	 0.36944011847178143
SWAR 	 0.35371426173618864
UPST 	 0.34726062226802745
MDB 	 0.3445228776121451
BKUS 	 0.3347140720912388
TSVT 	 0.3194444444444444
BNTX 	 0.2974041984756149


In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS